In [1]:
# -------------------------------
# Day 3: Imputation & Cleaning + Outlier & Duplicate Handling
# -------------------------------

import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

print("✅ Libraries loaded")

# -------------------------------
# Step 1: Load datasets
# -------------------------------
train_path = "../data/raw/train.csv"
anomaly_path = "../data/metadata/anomaly_flags.csv"

train_df = pd.read_csv(train_path)
anomaly_flags = pd.read_csv(anomaly_path)

print("Train shape:", train_df.shape)
print("Anomaly shape:", anomaly_flags.shape)

# -------------------------------
# Step 2: Merge anomaly flags
# -------------------------------
if "PassengerId" in train_df.columns and "PassengerId" in anomaly_flags.columns:
    merged_df = train_df.merge(anomaly_flags, on="PassengerId", how="left")
else:
    raise KeyError("❌ PassengerId column missing in either train.csv or anomaly_flags.csv")

print("Merged shape:", merged_df.shape)
print("Columns after merge:", merged_df.columns.tolist())

# -------------------------------
# Step 3: Handle missing values
# -------------------------------
num_cols = merged_df.select_dtypes(include=["int64", "float64"]).columns
cat_cols = merged_df.select_dtypes(include=["object"]).columns

# Mean imputation for numeric
num_imputer = SimpleImputer(strategy="mean")
merged_df[num_cols] = num_imputer.fit_transform(merged_df[num_cols])

# Mode imputation for categorical
cat_imputer = SimpleImputer(strategy="most_frequent")
merged_df[cat_cols] = cat_imputer.fit_transform(merged_df[cat_cols])

print("✅ Missing values imputed!")

# -------------------------------
# Step 4: Duplicate removal
# -------------------------------
duplicates = merged_df.duplicated()
print(f"⚠ Found {duplicates.sum()} duplicate rows")

merged_df = merged_df.drop_duplicates()
print("✅ Duplicates removed. New shape:", merged_df.shape)

# -------------------------------
# Step 5: Outlier Treatment & Logging
# -------------------------------
outlier_cols = [col for col in ["outlier_zscore", "outlier_iforest", "outlier_lstm"] if col in merged_df.columns]

if not outlier_cols:
    print("⚠ No outlier flag columns found — skipping outlier treatment.")
else:
    outlier_summary = {}
    for flag_col in outlier_cols:
        flagged = merged_df[merged_df[flag_col] == 1]
        outlier_summary[flag_col] = len(flagged)
        print(f"⚠ Found {len(flagged)} outliers flagged by {flag_col}")

    # Cap numeric outliers at 1st–99th percentiles
    for col in num_cols:
        lower, upper = np.percentile(merged_df[col], [1, 99])
        merged_df[col] = np.clip(merged_df[col], lower, upper)
    print("✅ Outliers capped at 1st–99th percentiles!")

# -------------------------------
# Step 6: Data Quality Report (Mini)
# -------------------------------
report = {
    "Missing values per column": merged_df.isnull().sum().to_dict(),
    "Duplicate rows": duplicates.sum(),
    "Outliers flagged": outlier_summary
}
report_path = "../data/processed/data_quality_report.txt"
os.makedirs(os.path.dirname(report_path), exist_ok=True)
with open(report_path, "w") as f:
    for k, v in report.items():
        f.write(f"{k}:\n{v}\n\n")

print(f"📝 Data quality report saved to {report_path}")

# -------------------------------
# Step 7: Save cleaned dataset
# -------------------------------
output_path = "../data/processed/train_clean.csv"
merged_df.to_csv(output_path, index=False)
print(f"🎯 Final cleaned dataset saved to {output_path}")
print("Final shape:", merged_df.shape)


✅ Libraries loaded
Train shape: (891, 12)
Anomaly shape: (891, 4)
Merged shape: (891, 15)
Columns after merge: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'outlier_zscore', 'outlier_iforest', 'outlier_lstm']
✅ Missing values imputed!
⚠ Found 0 duplicate rows
✅ Duplicates removed. New shape: (891, 15)
⚠ Found 65 outliers flagged by outlier_zscore
⚠ Found 45 outliers flagged by outlier_iforest
⚠ Found 45 outliers flagged by outlier_lstm
✅ Outliers capped at 1st–99th percentiles!
📝 Data quality report saved to ../data/processed/data_quality_report.txt
🎯 Final cleaned dataset saved to ../data/processed/train_clean.csv
Final shape: (891, 15)
